In [1]:
from datasets import load_dataset, Dataset
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer, sample_dataset

2023-05-25 13:52:34.666472: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 13:52:41.762535: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-25 13:52:41.762691: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-25 13:52:41.762705: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
# make the database object compatible with the model

def make_huggingface_dataset(filename):
    readfile = open(filename, 'r')
    addresses = []
    labels = []
    for line in readfile:
        addresses.append(line.split(',')[0].strip())
        labels.append(int(line.split(',')[1].strip()))

    # Create a Dataset object
    dataset = Dataset.from_dict({"idx": range(len(addresses)), "sentence": addresses, "label": labels})

    dset = Dataset.from_list(dataset)
    
    return dset

#these files (train.csv, eval.csv) are generated by skweak
t_dset = make_huggingface_dataset('train.csv')
e_dset = make_huggingface_dataset('eval.csv')



In [3]:
t_dset

Dataset({
    features: ['idx', 'sentence', 'label'],
    num_rows: 1754
})

In [4]:
e_dset

Dataset({
    features: ['idx', 'sentence', 'label'],
    num_rows: 439
})

In [5]:
# Load a SetFit model from Hub
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=t_dset,
    eval_dataset=e_dset,
    loss_class=CosineSimilarityLoss,
    metric="accuracy",
    batch_size=16,
    num_iterations=2, # The number of text pairs to generate for contrastive learning
    num_epochs=1, # The number of epochs to use for contrastive learning
    column_mapping={"sentence": "text", "label": "label"} # Map dataset columns to text/label expected by trainer
)

# Train and evaluate
model = trainer.train()

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Applying column mapping to training dataset
***** Running training *****
  Num examples = 7016
  Num epochs = 1
  Total optimization steps = 439
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/439 [00:00<?, ?it/s]

In [6]:
trainer.evaluate()

Applying column mapping to evaluation dataset
***** Running evaluation *****


{'accuracy': 0.9886104783599089}